In [1]:
from  sklearn.datasets import make_regression
from matplotlib import pyplot
import pandas as pd    
import numpy as np
from DGen import DGenerator

In [14]:
df_cadre = DGenerator.generate_cadres(10, 10, 5, 5, 101)
np.array(df_cadre['cadre'])

array([0, 0, 1, 1, 1, 2, 2, 3, 3, 3])

In [12]:
df_variables = DGenerator.generate_variables(df_cadre, 10, 5, 2, 3, 10)
df_variables

,dv0,dv1,dv2,dv3,dv4,target,cadre,index_c
0,2.334594,-1.461914,-1.698423,2.336225,2.416491,0,0,0
1,-2.114044,2.039460,1.854980,-1.164123,1.842182,1,0,1
2,-0.857185,0.288061,0.706234,-0.735791,-1.026204,0,0,2
3,0.608573,-0.323932,-0.905902,-0.461813,-2.244908,0,0,3
4,-0.543823,0.762009,0.269228,-0.718956,-0.238280,1,0,4
5,1.528598,-3.161474,-0.331722,2.723551,0.664783,0,0,5
6,-0.911076,-2.195991,1.863004,1.105104,-0.649393,1,0,6
7,0.110839,0.255052,-0.565296,-0.932984,-2.170261,0,0,7
8,-2.870849,2.951982,2.200476,-2.275953,0.915428,1,0,8
9,-0.136675,0.612532,-0.377733,-1.104456,-1.824018,1,0,9


,dv0,dv1,dv2,dv3,dv4,dv5,dv6,dv7,dv8,dv9,target,cadre,index_c
0,0.551890,0.502965,0.008838,1.275570,-0.636291,0.204991,-0.641438,0.011327,-1.881832,1.441763,0,0,0
1,1.664670,-0.334373,1.051210,-0.195847,1.510383,1.235555,0.509677,-0.904491,-1.617729,1.971669,0,0,1
2,0.679392,0.816741,0.820792,1.153407,0.222338,1.274302,-0.103165,0.694204,-1.185214,1.335822,0,0,2
3,0.588554,1.387142,0.646103,-1.776500,0.348179,0.288851,0.081470,0.387131,-0.260248,1.635921,0,0,3
4,1.925360,-3.378714,0.660975,-0.377428,-0.445843,1.497915,-3.341504,-0.871997,0.214619,1.725244,0,0,4
5,0.839804,-1.293716,-0.217751,0.827017,-0.808035,1.694055,-0.560068,-0.161056,-2.625815,0.491536,0,0,5
6,1.958058,0.182834,-0.661467,-1.818370,0.917387,0.093787,1.077235,-1.748909,-0.467890,2.709157,0,0,6
7,1.094342,0.247200,0.610592,-1.842442,0.684532,1.605830,-0.589799,0.423374,-1.972835,1.775515,0,0,7
8,2.539283,-2.067085,0.134627,1.067499,-0.897182,3.170502,-0.577288,-0.217005,3.025278,1.881224,1,0,8
9,-0.759844,-0.138870,0.289948,-0.490814,0.662827,2.268830,0.536834,1.855631,-0.362927,-1.923374,1,0,9


In [ ]:
X = 